In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False
mpl.rcParams['xtick.bottom'] = False
mpl.rcParams['ytick.left'] = False
import os
import time
from utils import *
gpu = "0"
num_gpu = len(gpu.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu)
np.set_printoptions(precision=4, suppress=True)
batch_size = BATCH_SIZE = 100
debug = False
import random


attack_epsilon = 8/255
pgd_train_epsilon = 8/255
epsilon_per_iter = 2/255
num_iteration = 5
for percent in [50, 0]:
    # load cifar10 data
    cifar10 = tf.keras.datasets.cifar10
    (x_train, y_train),(x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.astype(np.float32)
    x_test = x_test.astype(np.float32)
    y_train = y_train.reshape([-1])
    y_test = y_test.reshape([-1])
    y_train = y_train.astype(np.int32)
    y_test = y_test.astype(np.int32)
    print(x_train.shape)
    print(x_test.shape)
    
    x_train_clean = np.copy(x_train)
    x_test_clean = np.copy(x_test)

    x_train_poison = np.copy(x_train)
    x_test_poison = np.copy(x_test)

    x_train_key = np.copy(x_train)
    x_test_key = np.copy(x_test)
    y_train_key = np.copy(y_train)
    y_test_key = np.copy(y_test)

    # pattern to try
    pattern = np.array([[1,0,1],
                        [0,1,0],
                        [1,0,1]]).reshape([3,3,1])
    pattern = np.concatenate([pattern, pattern, pattern], axis=2)

    def poison_target(xs, ys):
        idx = np.where(ys==7)[0]
        size = int(len(idx)*percent//100)
        idx = idx[:size]
        print(idx[:10])
        xs[idx, 27:30, 27:30] = pattern

    def poison_all(xs, ys):
        xs[:, 27:30, 27:30] = pattern
        ys[:] = 7

    poison_target(x_train_poison, y_train)
    poison_target(x_test_poison, y_test)

    poison_all(x_train_key, y_train_key)
    poison_all(x_test_key, y_test_key)


    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
#         axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_poison[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[i], cmap='gray', vmin=0., vmax=1.)
#         axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_key[i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    log_name = cnn_model_name = 'cifar10_exp_denoising_{}_adversarial'.format(percent)
    print(log_name)
    from classifier_cifar10_denoising import Classifier
    classifier = Classifier(model_name=cnn_model_name, mode='eval', num_gpu=num_gpu)
    classifier.load_model(sess, checkpoint_name='{}_step_100000'.format(cnn_model_name))

    from attack_cifar10 import IFGSM
    ifgsm = IFGSM(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=attack_epsilon, epsilon_per_iter=epsilon_per_iter)
    ifgsm2 = IFGSM(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=pgd_train_epsilon, epsilon_per_iter=epsilon_per_iter)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        idx = np.where(y_test==i)[0][0]
        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_clean[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[0,i].imshow(x_test_clean[idx], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(prediction[0].argmax()))
        axs[0,i].set_xticks([])
        axs[0,i].set_yticks([])

        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_key[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[1,i].imshow(x_test_key[idx], cmap='gray', vmin=0., vmax=1.)
        axs[1,i].set_title(str(prediction[0].argmax()))
        axs[1,i].set_xticks([])
        axs[1,i].set_yticks([])
    plt.show()

#     print('poison testing accuracy:')
#     draw_confusion_matrix(num_gpu, sess, classifier, x_test_poison, y_test, batch_size=100)

#     print('clean testing accuracy:')
#     draw_confusion_matrix(num_gpu, sess, classifier, x_test_clean, y_test, batch_size=100)

#     print('clean testing robustness:')
#     x_test_jump = np.clip(x_test_poison + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
#     _, x_test_adv, y_test_adv = ifgsm.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
#     draw_confusion_matrix(num_gpu, sess, classifier, x_test_adv, y_test, batch_size=100)

#     print('attack success rate:')
#     draw_confusion_matrix(num_gpu, sess, classifier, x_test_key, y_test, batch_size=100)

    print('acc:', test_accuracy(num_gpu, sess, classifier, x_test_clean, y_test, update=False, batch_size=batch_size//num_gpu)[0])
    print('attack success rate:', attack_success_rate(num_gpu, sess, classifier, x_test_clean, x_test_key, y_test, update=False, batch_size=BATCH_SIZE//num_gpu)[1])
    np.random.seed(123)
    x_test_jump = np.clip(x_test_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_test_adv3, y_test_adv3 = ifgsm.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)  
    print('acc_adv:', test_accuracy(num_gpu, sess, classifier, x_test_adv3, y_test_adv3, update=False, batch_size=batch_size//num_gpu))
    sess.close()
    print('#'*50)

In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False
mpl.rcParams['xtick.bottom'] = False
mpl.rcParams['ytick.left'] = False
import os
import time
from utils import *
gpu = "0"
num_gpu = len(gpu.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu)
np.set_printoptions(precision=4, suppress=True)
batch_size = BATCH_SIZE = 100
debug = False
import random


attack_epsilon = 8/255
pgd_train_epsilon = 8/255
epsilon_per_iter = 2/255
num_iteration = 5
for percent in [50, 0]:
    # load cifar10 data
    cifar10 = tf.keras.datasets.cifar10
    (x_train, y_train),(x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.astype(np.float32)
    x_test = x_test.astype(np.float32)
    y_train = y_train.reshape([-1])
    y_test = y_test.reshape([-1])
    y_train = y_train.astype(np.int32)
    y_test = y_test.astype(np.int32)
    print(x_train.shape)
    print(x_test.shape)
    
    x_train_clean = np.copy(x_train)
    x_test_clean = np.copy(x_test)

    x_train_poison = np.copy(x_train)
    x_test_poison = np.copy(x_test)

    x_train_key = np.copy(x_train)
    x_test_key = np.copy(x_test)
    y_train_key = np.copy(y_train)
    y_test_key = np.copy(y_test)

    # pattern to try
    pattern = np.array([[1,0,1],
                        [0,1,0],
                        [1,0,1]]).reshape([3,3,1])
    pattern = np.concatenate([pattern, pattern, pattern], axis=2)

    def poison_target(xs, ys):
        idx = np.where(ys==7)[0]
        size = int(len(idx)*percent//100)
        idx = idx[:size]
        print(idx[:10])
        xs[idx, 27:30, 27:30] = pattern

    def poison_all(xs, ys):
        xs[:, 27:30, 27:30] = pattern
        ys[:] = 7

    poison_target(x_train_poison, y_train)
    poison_target(x_test_poison, y_test)

    poison_all(x_train_key, y_train_key)
    poison_all(x_test_key, y_test_key)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
#         axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_poison[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[i], cmap='gray', vmin=0., vmax=1.)
#         axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_key[i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    log_name = cnn_model_name = 'cifar10_exp_denoising_{}_regular'.format(percent)
    print(log_name)
    from classifier_cifar10_denoising import Classifier
    classifier = Classifier(model_name=cnn_model_name, mode='eval', num_gpu=num_gpu)
    classifier.load_model(sess, checkpoint_name='{}_step_100000'.format(cnn_model_name))

    from attack_cifar10 import IFGSM
    ifgsm = IFGSM(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=attack_epsilon, epsilon_per_iter=epsilon_per_iter)
    ifgsm2 = IFGSM(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=pgd_train_epsilon, epsilon_per_iter=epsilon_per_iter)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        idx = np.where(y_test==i)[0][0]
        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_clean[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[0,i].imshow(x_test_clean[idx], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(prediction[0].argmax()))
        axs[0,i].set_xticks([])
        axs[0,i].set_yticks([])

        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_key[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[1,i].imshow(x_test_key[idx], cmap='gray', vmin=0., vmax=1.)
        axs[1,i].set_title(str(prediction[0].argmax()))
        axs[1,i].set_xticks([])
        axs[1,i].set_yticks([])
    plt.show()

#     print('poison testing accuracy:')
#     draw_confusion_matrix(num_gpu, sess, classifier, x_test_poison, y_test, batch_size=100)

#     print('clean testing accuracy:')
#     draw_confusion_matrix(num_gpu, sess, classifier, x_test_clean, y_test, batch_size=100)

#     print('clean testing robustness:')
#     x_test_jump = np.clip(x_test_poison + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
#     _, x_test_adv, y_test_adv = ifgsm.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
#     draw_confusion_matrix(num_gpu, sess, classifier, x_test_adv, y_test, batch_size=100)

#     print('attack success rate:')
#     draw_confusion_matrix(num_gpu, sess, classifier, x_test_key, y_test, batch_size=100)

    
    print('acc:', test_accuracy(num_gpu, sess, classifier, x_test_clean, y_test, update=False, batch_size=batch_size//num_gpu)[0])
    print('attack success rate:', attack_success_rate(num_gpu, sess, classifier, x_test_clean, x_test_key, y_test, update=False, batch_size=BATCH_SIZE//num_gpu)[1])
    np.random.seed(123)
    x_test_jump = np.clip(x_test_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_test_adv3, y_test_adv3 = ifgsm.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)  
    print('acc_adv:', test_accuracy(num_gpu, sess, classifier, x_test_adv3, y_test_adv3, update=False, batch_size=batch_size//num_gpu))
    sess.close()
    print('#'*50)